# Recovering Gauss coefficients by grid search

In this notebook, we compute the posterior distribution for the geomagnetic problem using a simple grid search. For this, we discretise the model space on a regular grid and evaluate the prior distributions for all trial models. Owing to the rapidly increasing number of trial models with increasing model space dimension, this is only computationally feasible in 1-D and 2-D.

# 0. Import some Python packages

We begin by importing some Python packages and for plotting.

In [ ]:
# Some Python packages.
import magnetic as magnetic
import random
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
import time

# Set some parameters to make plots nicer.

plt.rcParams["font.family"] = "serif"
plt.rcParams.update({'font.size': 25})

# 1. Initialisations

Read the Gauss coefficients from the IGRF13 model. These will be used as ground-thruth parameters that we try to estimate.

In [ ]:
# Read Gauss coefficients from IGRF13.
g_igrf13,h_igrf13=magnetic.read_coefficients()

For convenience, we define two functions that implement the priors in data and in model space. This is done in magnetic.py, which we imported above.

**Exercise 1**: Plot the data prior prior_data( ) for $d^{obs}=0$.

The key prior information on the spherical harmonics coefficients is that the energy of the magnetic field must be finite. The energy $E$ is equal to the squared sum of all coefficients, that is, $\sum_{\ell=0}^\infty \sum_{m=0}^\ell [(g_\ell^m)^2+(h_\ell^m)^2]$. $$ $$

**Exercise 2**: Plot the model space prior prior_model( ) as a function of the energy $E$.

We define the colatitude and longitude of the observation points.

In [ ]:
# Observation points.

# Example of just one observation point.
theta_obs=np.array([0.2])
phi_obs=np.array([0.149])

# Example with three observation points.
#theta_obs=np.array([0.2, 0.7, 1.5])
#phi_obs=np.array([0.149, 1.0, 2.0])

In [ ]:
# Precompute associated Legendre functions to accelerate computations.
Pnmi=magnetic.Pnmi(theta_obs,ell_max=1)

# 2. Grid search in 1D

We begin by trying to estimate only one model parameter, namely $g_1^0$. For this, we produce artificial observations by solving the forward problem with only $g_1^0 \neq 0$.

In [ ]:
# Make the ground-truth model by reading Gauss coefficients from IGRF13 and using only the first one.
g=np.zeros(np.shape(g_igrf13))
h=np.zeros(np.shape(g_igrf13))
g[1,0]=g_igrf13[1,0]

# Compute the magnetic field values for the observation points.
d_obs=magnetic.B_field(phi_obs,theta_obs,g,h).diagonal()

The next code block plots the magnetic field and the observation points on a map.

In [ ]:
# Compute magnetic field for longitude and colatitude arrays.
theta=np.arange(0.0,np.pi,0.05)
phi=np.arange(0.0,2.0*np.pi,0.05)

d_plot=magnetic.B_field(phi,theta,g,h,ell_max=1)
t2=time.time()

# Plot radial component of the magnetic field.
colat,lon=np.meshgrid(phi,theta)

plt.subplots(1, figsize=(22,10))
plt.gca().invert_yaxis()
plt.pcolor(180.0*colat/np.pi,180.0*lon/np.pi,d_plot, cmap=plt.cm.get_cmap('Greys'))
plt.colorbar()
plt.contour(180.0*colat/np.pi,180.0*lon/np.pi,d_plot, colors='k')
plt.plot(180.0*phi_obs/np.pi,180.0*theta_obs/np.pi,'ro',markersize=10)
plt.grid()
plt.xlabel('longitude [°]',labelpad=15)
plt.ylabel('colatitude [°]',labelpad=15)
plt.title('magnetic field, radial component',pad=20)
plt.show()

Based on these preparations, we can now run a 1-D grid search by simply marching through a pre-defined array for $g_1^0$ test values.

**Exercise 3**: Sample the posterior by running the code below and discuss the results. (What is the value of the maximum-likelihood coefficient? Provide a visual guess of the standard deviation. Does the posterior have more than one peak?)

**Exercise 4**: Repeat the probabilistic inversion with just one observation point. How does the reduction of observation points affect the quality of your inference?

**Exercise 5**: Repeat the inversion with 20 observation points. How does the increase of observation points affect the quality of your inference?

In [ ]:
# Initialise the values for which to evaluate the posterior.
g_test=np.zeros(np.shape(g_igrf13))
h_test=np.zeros(np.shape(h_igrf13))

dg=200.0
g10=np.arange(-33000.0,-25000.0,dg)
posterior=np.zeros(len(g10))

t1=time.time()

# March through all test values and evaluate the posterior by computing test data.
for i in range(len(g10)):
    
    g_test[1,0]=g10[i]
    
    d=magnetic.B(phi_obs,theta_obs,g_test,h_test,Pnmi,ell_max=1)
    posterior[i]=magnetic.prior_model(g_test,h_test,ell_max=1)*magnetic.prior_data(d,d_obs)
   
t2=time.time()
print('elapsed time: %f s' % (t2-t1))

# Normalise the posterior.
normalisation=np.sum(posterior)*dg
posterior=posterior/normalisation

# Plot the posterior.   
plt.subplots(1, figsize=(10,10))
plt.plot(g10,posterior,'k',linewidth=2)
plt.xlim((g10[0],g10[-1]))
plt.xlabel(r'$g_1^0$ [nT]',labelpad=15)
plt.ylabel('posterior',labelpad=15)
plt.title('posterior probability density',pad=15)
plt.grid()
plt.show()

# 3. Grid search in 2D

Now we repeat the same for two non-zero model parameters, $g_1^0$ and $g_1^1$. First, we compute the artificial observations.

In [ ]:
# Make the ground-truth model. We choose only two Gauss coefficients from IGRF13.
g=np.zeros(np.shape(g_igrf13))
h=np.zeros(np.shape(g_igrf13))
g[1,0]=g_igrf13[1,0]
g[1,1]=g_igrf13[1,1]

# Compute the magnetic field values for the observation points.
d_obs=magnetic.B(phi_obs,theta_obs,g,h,Pnmi,ell_max=1)

The next code block plots the magnetic field and the observation points on a map.

In [ ]:
# Compute magnetic field for longitude and colatitude arrays.
theta=np.arange(0.0,np.pi,0.05)
phi=np.arange(0.0,2.0*np.pi,0.05)

d_plot=magnetic.B_field(phi,theta,g,h,ell_max=1)

# Plot radial component of the magnetic field.
colat,lon=np.meshgrid(phi,theta)

plt.subplots(1, figsize=(22,10))
plt.gca().invert_yaxis()
plt.pcolor(180.0*colat/np.pi,180.0*lon/np.pi,d_plot, cmap=plt.cm.get_cmap('Greys'))
plt.colorbar()
plt.contour(180.0*colat/np.pi,180.0*lon/np.pi,d_plot, colors='k')
plt.plot(180.0*phi_obs/np.pi,180.0*theta_obs/np.pi,'ro',markersize=10)
plt.grid()
plt.xlabel('longitude [°]',labelpad=15)
plt.ylabel('colatitude [°]',labelpad=15)
plt.title('magnetic field, radial component',pad=20)
plt.show()

Similar to the 1-D grid search, we now march through a 2-D array of pre-defined test values for $g_1^0$ and $g_1^1$.

**Exercise 6**: Using the same data and model priors as before, we evaluate the posterior via a systematic grid search of all plausible values for $g_1^0$ and $g_1^1$. For this, run the code below and analyse the posterior distribution.

**Exercise 7**: Repeat the inversion with only one observation point. Can the two coefficients still be constrained independently? Compare your results with those of your colleagues.

**Exercise 8**: How much time is needed to invert for the 2 coefficients with 3 observation points? How does this time compare to the inversion for just 1 coefficient? For how many parameters do you think you can invert in less than 1 hour? Is grid search a feasible approach to the solution of large probabilistic inverse problems?

In [ ]:
# Initialise the values for which to evaluate the posterior.
g_test=np.zeros(np.shape(g_igrf13))
h_test=np.zeros(np.shape(h_igrf13))

dg=100.0
g10=np.arange(-33000.0,-20000.0,dg)
g11=np.arange(-15000.0,26000.0,dg)
posterior=np.zeros((len(g10),len(g11)))

t1=time.time()

# March through all test values and evaluate the posterior by computing test data.
for i in range(len(g10)):
    for j in range(len(g11)):
        
        g_test[1,0]=g10[i]
        g_test[1,1]=g11[j]
        
        d=magnetic.B(phi_obs,theta_obs,g_test,h_test,Pnmi,ell_max=1)
        posterior[i,j]=magnetic.prior_model(g_test,h_test,ell_max=1)*magnetic.prior_data(d,d_obs)

t2=time.time()
print('elapsed time: %f s' % (t2-t1))
        
# Normalise the posterior.
normalisation=np.sum(posterior)*dg*dg
posterior=posterior/normalisation
        
# Plot.
plt.subplots(1, figsize=(10,10))
g10,g11=np.meshgrid(g11,g10)
plt.pcolor(g11,g10,posterior, cmap=plt.cm.get_cmap('binary'))
plt.xlabel(r'$g_1^0$ [nT]',labelpad=15)
plt.ylabel(r'$g_1^1$ [nT]',labelpad=15)
plt.colorbar()
plt.title('posterior probability density',pad=15)
plt.grid()
plt.show()

# 4. Spectral leakage

The real magnetic field has, of course, infinitely many spherical harmonic coefficients. However, since our computational resources are finite, we have to truncate the expansion after a finite number of coefficients. The neglect of higher-order coefficients tends to pollute the lower-order coefficients that we wish to infer as accurately as possible. This phenomenon is referred to as *spectral leakage*. (The higher-order coefficients leak into the lower-order coefficients.)

**Optional exercise 9**: Generate synthetic data for the complete set of coefficients from the IGRF13 model. Then invert these data under the assumption that only $g_1^0$ is different from $0$. Is it possible to obtain reliable results? $$ $$